<a href="https://colab.research.google.com/github/DonJM/FCLB-TFOD/blob/master/FCLB_TFOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

#import neccessary python modules and libraries
import os
import shutil
from PIL import ImageEnhance
from PIL import Image
import uuid
import tarfile
import wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=506d4b1178488601069f9858af8974a642a603c76651635112e0aa8af8e6fc9a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


# Adding Classes for Path Setup and Project Dataset

In [90]:
class Setup_Path: 

    LABELS = ["fresh", "rotten"]

    def getFiles(self):
      files = {
          "LABELMAP":os.path.join(self.getWorkspaces()['ANNOTATIONS_PATH'], "label_map.pbtxt"),
          'TFRECORD_SCRIPT':os.path.join(self.getWorkspaces()['PREPROCESSING_PATH'], 'generate_tfrecord.py'),
          "PRETRAINED_MODEL_NAME": 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
          "PRETRAINED_MODEL_URL": 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz', 
      }

      return files

    def getWorkspaces(self):
        paths = {
            "ADDONS_PATH":os.path.join("Tensorflow", "addons"),
            "LABEL_IMG_PATH":os.path.join("Tensorflow", "addons", "labelImg"),
            "TFOD_API_PATH":"Tensorflow",
            "WORKSPACE_PATH":os.path.join("Tensorflow", "workspace"),
            "TRAINING_DEMO_PATH":os.path.join("Tensorflow", "workspace", "training_demo"),
            "ANNOTATIONS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "annotations"),
            "EXPORTED_MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "exported-models"),
            "IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images"),
            "TRAINING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "train"),
            "TESTING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "test"),
            "MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo"),
            "PRETRAINED_MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "pre-trained-models"),
            "PREPROCESSED_IMG":os.path.join("Tensorflow", "addons", "preprocessedImg"),
            "ORIGINAL_IMGS":os.path.join("Tensorflow", "addons", "originalImg"),   
            "PREPROCESSING_PATH":os.path.join("Tensorflow", "scripts", "preprocessing"),
            "PROTOS_PATH":os.path.join("Tensorflow", "Google-Protobuf"),
            'CHECKPOINT_PATH': os.path.join("Tensorflow", "workspace", "training_demo", 'my_ssd_mobnet')
        }

        return paths

    # def set_main_paths(self):
    #     for path in self.get_main_paths().values():
    #         if not os.path.exists(path):
    #             os.makedirs(path)

    # def get_image_paths(self):
    #     path = {
    #         "ORIGINAL_IMAGES_PATH" : ["workspace\datasets\original_images\Fresh_Ripe", "workspace\datasets\original_images\Fresh_Unripe", "workspace\datasets\original_images\Rotten_Ripe", "workspace\datasets\original_images\Rotten_Unripe"],
    #         "PREPROCESSED_IMAGES_PATH" : ["workspace\datasets\preprocessed_images\Fresh_Ripe", "workspace\datasets\preprocessed_images\Fresh_Unripe", "workspace\datasets\preprocessed_images\Rotten_Ripe", "workspace\datasets\preprocessed_images\Rotten_Unripe"],
    #     }

    #     return path

    # def set_image_paths(self):
    #     for path in self.get_image_paths().values():
    #         for label in path:
    #             if not os.path.exists(label):
    #                 os.makedirs(label)

    # def getFiles(self):
    #     files = {
    #         'PIPELINE_CONFIG':os.path.join(self.get_paths()['MODELS_PATH'], self.custom_name, 'pipeline.config'),
    #         'TF_RECORD_SCRIPT':os.path.join(self.get_paths()['TFRECORD_GENERATOR_PATH'], self.tfrecord_name),
    #         'LABEL_MAP':os.path.join(self.get_paths()['ANOTATIONS_PATH'], self.map_name),
    #     }
    #     return files

In [3]:
class FCLB_Datasets:
    #class' constructor
    def __init__(self, setup):
        self.setup = setup
    
    #acquiring uncategorized dataset
    def get_original_dataset(self, num1, num2, label):
        images_path = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        path = os.path.join(images_path, label)

        files = ["{}".format(i) for i in os.listdir(path)[num1:num2]]

        return files

    def copy_image_file(self, num1, num2, srcPath, dstPath):

        for label in self.setup.LABELS:
            for file in self.get_original_dataset(num1, num2, label):
                dst = os.path.join(dstPath, label, file)
                src = os.path.join(srcPath, label, file)
                shutil.copyfile(src, dst)

    #I call it segment function, wala na koy mahunahunaan
    def segment(self):
        original_dir = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        target_dir = self.setup.getWorkspaces()['PREPROCESSED_IMG']
        labels = self.setup.LABELS

        for label in labels:
            original = os.path.join(original_dir, label)
            target = os.path.join(target_dir, label)
            for img in os.listdir(original):
                self.segmentation(original, target, img, label)
 

    #this function used to segment an image file, it actually resize image, add contrast and lightness
    #and save it to the target directory
    #the purpose of uuid module is to give unique id or name to an image file
    def segmentation(self, base_dir, target_dir, fname, label):
        image = Image.open(os.path.join(base_dir, fname))
        size = (400,400)
        image.thumbnail(size)

        contrast = ImageEnhance.Contrast(image)
        contrast.enhance(1.8).save(os.path.join(target_dir, label + "-"+ '{}.jpg'.format(uuid.uuid1())))

        brightness = ImageEnhance.Brightness(image)
        brightness.enhance(1.2).save(os.path.join(target_dir, label +"-"+ '{}.jpg'.format(uuid.uuid1())))

    #compressed file for github upload
    def compressed(self):
        path = self.getDir('PREPROCESSED_IMAGES_PATH')
        sub_paths = self.getSubDirectories(path)
        ARCHIVE_PATH = os.path.join(path, 'preprocessed_images.tar.gz')
        tar = tarfile.open(ARCHIVE_PATH, "w:gz")
        for sub_path in sub_paths:
            dir = os.path.join(path, sub_path)
            tar.add(dir)
        tar.close()

# Preparing Workspace

Creating a workspace directory

In [91]:
setup = Setup_Path()
workspaces = setup.getWorkspaces()
for workspace in workspaces.values():
  if not os.path.exists(workspace):
    os.makedirs(workspace)

In [48]:
!unzip /content/train.zip
shutil.move('/content/fresh', setup.getWorkspaces()['ORIGINAL_IMGS'])
shutil.move('/content/rotten', setup.getWorkspaces()['ORIGINAL_IMGS'])

Archive:  /content/train.zip
replace fresh/Fresh.526.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fresh/Fresh.534.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace fresh/Fresh.534.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.534.jpg     
replace fresh/Fresh.718.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.718.jpg     
replace fresh/Fresh.729.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.729.jpg     
replace fresh/Fresh.743.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.743.jpg     
replace fresh/Fresh.799.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.799.jpg     
replace rotten/Rotten.120.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rotten/Rotten.120.jpg   
replace rotten/Rotten.1538.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rotten/Rotten.1538.jpg  
replace rotten/Rotten.1553.jpg? [y]es, [n]o, [A]ll

'Tensorflow/addons/originalImg/rotten'

Create label directory for original images and preprocessed images

In [49]:
for label in setup.LABELS:
  preprocessedImg_labelPath = os.path.join(setup.getWorkspaces()['PREPROCESSED_IMG'], label)
  if not os.path.exists(preprocessedImg_labelPath):
    os.makedirs(preprocessedImg_labelPath)

segment images and copy to preprocessed directory

In [50]:
dataset = FCLB_Datasets(setup)
dataset.segment()

In [64]:
!cd /content/Tensorflow && zip {setup.getWorkspaces()['PREPROCESSED_IMG']}


zip error: Nothing to do! (Tensorflow/addons/preprocessedImg.zip)


# Image Labelling

In [ ]:
labelImg = setup.getWorkspaces()['LABEL_IMG_PATH']
!cd {labelImg} && git clone https://github.com/tzutalin/labelImg.git

Cloning into 'labelImg'...
remote: Enumerating objects: 1846, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1846 (delta 37), reused 55 (delta 18), pack-reused 1759
Receiving objects: 100% (1846/1846), 232.81 MiB | 41.39 MiB/s, done.
Resolving deltas: 100% (1084/1084), done.


In [ ]:
labelImg = os.path.join(setup.getWorkspaces()['LABEL_IMG_PATH'], "labelImg")
!cd {labelImg} && pip install pyqt5
!cd {labelImg} && pyrcc5 -o libs/resources.py resources.qrc

# Partition Dataset

In [ ]:
url = "https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/d0e545609c5f7f49f39abc7b6a38cec3/partition_dataset.py"
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/partition_dataset.py'

In [ ]:
images_path = setup.getWorkspaces()['IMAGES_PATH']
!cd {dst} python partition_dataset.py -x -i {images_path} -r 0.1

/bin/bash: line 0: cd: too many arguments


# Download Tensorflow Object Detection API

In [5]:
tfod_api = setup.getWorkspaces()['TFOD_API_PATH']
!cd {tfod_api} && git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 61038, done.
remote: Total 61038 (delta 0), reused 0 (delta 0), pack-reused 61038
Receiving objects: 100% (61038/61038), 573.97 MiB | 31.64 MiB/s, done.
Resolving deltas: 100% (42534/42534), done.


Install Object Detection

In [30]:
research_path = os.path.join(setup.getWorkspaces()['TFOD_API_PATH'], 'models', 'research')
!cd {research_path} && cp object_detection/packages/tf2/setup.py setup.py

In [6]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
src = wget.download(url)
protos_path = setup.getWorkspaces()['PROTOS_PATH']
shutil.move(src, protos_path)

'Tensorflow/Google-Protobuf/protoc-3.15.6-win64.zip'

In [98]:
research_path = os.path.join(setup.getWorkspaces()['TFOD_API_PATH'], 'models', 'research')
slim_path = os.path.join(research_path, 'slim')
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(protos_path, 'bin'))   
!cd {research_path} && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py setup.py && python setup.py build && python setup.py install
!cd {slim_path} && pip install -e . 

running build
running build_py
copying object_detection/protos/ssd_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/center_net_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/string_int_label_map_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/train_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/multiscale_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/flexible_grid_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/pipeline_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/prot

In [99]:
# !cd {research_path} && python object_detection/builders/model_builder_tf2_test.py
VERIFICATION_SCRIPT = os.path.join(research_path, 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-09-01 05:39:16.970220: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-01 05:39:16.970275: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feb5f230b6c9): /proc/driver/nvidia/version does not exist
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0901 05:39:17.257028 140693938935680 model_builder.py:1088] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.71s
I0901 05:39:17.680129 140693938935680 test_util.py:2189] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.71s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorf

In [110]:
!pip install tensorflow --upgrade

In [120]:
!pip install imgaug>=0.2.5
!pip install pyyaml>=5.1
!pip install avro-python3>=1.8.1
!pip install dill>=0.3.1.1
!pip install future>=0.18.2
!pip install requests>=2.24.0

In [121]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib>=3.2

Found existing installation: protobuf 3.17.3
Uninstalling protobuf-3.17.3:
  Successfully uninstalled protobuf-3.17.3
Found existing installation: matplotlib 3.4.3
Uninstalling matplotlib-3.4.3:
  Successfully uninstalled matplotlib-3.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
tf-models-official 2.6.0 requires pyyaml>=5.1, but you have pyyaml 3.13 which is incompatible.
apache-beam 2.32.0 requires avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you have avro-python3 1.10.2 which is incompatible.
apache-beam 2.32.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.4 which is incompatible.
apache-beam 2.32.0 requires future<1.0.0,>=0.18.2, but you have future 0.16.0 which is incompatible.
apache-beam 2.32.0 requires requests<3.0.0,>=2.24.0, but you have requ

In [122]:
import object_detection

ModuleNotFoundError: ignored

# Create Label Map

In [58]:
labels = [{'name':'fresh', 'id':1}, {'name':'rotten', 'id':2}]

with open(setup.getFiles('ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',  'my_ssd_mobnet')['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create Tensorflow Records

In [52]:
url = 'https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py'
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/generate_tfrecord.py'

In [53]:
# Create train data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TRAINING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

# Create test data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TESTING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

yy
y
2021-09-01 04:33:04.066910: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/training_demo/annotations/train.record
2021-09-01 04:33:07.017832: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/training_demo/annotations/train.record


# Download Pretrained Model

In [75]:
pretrained_mode_file = wget.download(setup.getFiles()['PRETRAINED_MODEL_URL'])
shutil.move(pretrained_mode_file, setup.getWorkspaces()['PRETRAINED_MODELS_PATH'])
shutil.move(pretrained_mode_file, setup.getWorkspaces()['PRETRAINED_MODELS_PATH'])

# Make yourown copy of pre-trained model

In [94]:
!cp {os.path.join(setup.getWorkspaces()['PRETRAINED_MODELS_PATH'], 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'pipeline.config')} {os.path.join(setup.getWorkspaces()['CHECKPOINT_PATH'])}

In [95]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ModuleNotFoundError: ignored